In [31]:
import os
import csv
import pandas as pd
# import numpy as pd




In [32]:
#http://localhost:8889/edit/Resources/nfl_stadiums.csv
file = "Resources/nfl_stadiums.csv"

In [33]:
file_df = pd.read_csv(file, encoding="ISO-8859-1")

In [34]:
file_df.head(32)

,stadium_name,stadium_location,stadium_open,stadium_close,stadium_type,stadium_address,stadium_weather_station_code,stadium_weather_type,stadium_capacity,stadium_surface,STATION,NAME,LATITUDE,LONGITUDE,ELEVATION
0,Alamo Dome,"San Antonio, TX",NaN,NaN,indoor,"100 Montana St, San Antonio, TX 78203",78203,dome,72000,FieldTurf,NaN,NaN,NaN,NaN,NaN
1,Allegiant Stadium,"Paradise, NV",2020.0,NaN,indoor,NaN,NaN,dome,65000,Grass,NaN,NaN,NaN,NaN,NaN
2,Alltel Stadium,"Jacksonville, FL",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Alumni Stadium,"Chestnut Hill, MA",NaN,NaN,outdoor,"Perimeter Rd, Chestnut Hill, MA 02467",2467,cold,NaN,Grass,NaN,NaN,NaN,NaN,NaN
4,Anaheim Stadium,"Anaheim, CA",1980.0,1994.0,outdoor,"2000 E Gene Autry Way, Anaheim, CA 92806",92806,warm,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Arrowhead Stadium,"Kansas City, MO",1972.0,NaN,outdoor,"1 Arrowhead Dr, Kansas City, MO 64129",64129,cold,"76,416",Grass,US1MOJC0028,"KANSAS CITY 5.1 SE, MO US",39.06920,-94.48710,264.9
6,AT&T Stadium,"Arlington, TX",2009.0,NaN,retractable,"1 AT&T Way, Arlington, TX 76011",76011,dome,"80,000",FieldTurf,USC00410337,"ARLINGTON SIX FLAGS, TX US",32.75720,-97.07360,163.4
7,Atlanta-Fulton County Stadium,"Atlanta, GA",1966.0,1991.0,outdoor,"521 Capitol Avenue SE, Atlanta, Georgia 30312",30312,warm,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Balboa Stadium,"San Diego, CA",NaN,NaN,outdoor,"Balboa Stadium, San Diego, CA 92101",92101,warm,NaN,Grass,NaN,NaN,NaN,NaN,NaN
9,Bank of America Stadium,"Charlotte, NC",1996.0,2016.0,outdoor,"800 S Mint St, Charlotte, NC 28202",28202,moderate,"73,778",Grass,USW00013881,"CHARLOTTE DOUGLAS AIRPORT, NC US",35.22360,-80.95520,221.9


In [35]:
file_df[["stadium_name","stadium_location","stadium_type","stadium_address","stadium_weather_type","stadium_capacity"]].head()

,stadium_name,stadium_location,stadium_type,stadium_address,stadium_weather_type,stadium_capacity
0,Alamo Dome,"San Antonio, TX",indoor,"100 Montana St, San Antonio, TX 78203",dome,72000
1,Allegiant Stadium,"Paradise, NV",indoor,NaN,dome,65000
2,Alltel Stadium,"Jacksonville, FL",NaN,NaN,NaN,NaN
3,Alumni Stadium,"Chestnut Hill, MA",outdoor,"Perimeter Rd, Chestnut Hill, MA 02467",cold,NaN
4,Anaheim Stadium,"Anaheim, CA",outdoor,"2000 E Gene Autry Way, Anaheim, CA 92806",warm,NaN


In [65]:
del file_df['stadium_weather_type']

In [110]:
file_df.head(10)

,stadium_name,stadium_location,stadium_type,stadium_capacity,stadium_surface
0,Alamo Dome,"San Antonio, TX",indoor,72000,FieldTurf
1,Allegiant Stadium,"Paradise, NV",indoor,65000,Grass
2,Alltel Stadium,"Jacksonville, FL",NaN,NaN,NaN
3,Alumni Stadium,"Chestnut Hill, MA",outdoor,NaN,Grass
4,Anaheim Stadium,"Anaheim, CA",outdoor,NaN,NaN
5,Arrowhead Stadium,"Kansas City, MO",outdoor,"76,416",Grass
6,AT&T Stadium,"Arlington, TX",retractable,"80,000",FieldTurf
7,Atlanta-Fulton County Stadium,"Atlanta, GA",outdoor,NaN,NaN
8,Balboa Stadium,"San Diego, CA",outdoor,NaN,Grass
9,Bank of America Stadium,"Charlotte, NC",outdoor,"73,778",Grass


In [111]:
df1= pd.DataFrame(file_df)

df1.to_csv(r'C:\Users\Stron\csv_files\df1.csv', index = False, header=True)

In [68]:
file2 = "Resources/nfl_teams.csv"

file_df2 = pd.read_csv(file2, encoding="ISO-8859-1")

file_df2.head(3)


,team_name,team_name_short,team_id,team_id_pfr,team_conference,team_division,team_conference_pre2002,team_division_pre2002
0,Arizona Cardinals,Cardinals,ARI,CRD,NFC,NFC West,NFC,NFC West
1,Phoenix Cardinals,Cardinals,ARI,CRD,NFC,NaN,NFC,NFC East
2,St. Louis Cardinals,Cardinals,ARI,ARI,NFC,NaN,NFC,NFC East


In [69]:
del file_df2['team_id_pfr']
del file_df2['team_name_short']
del file_df2['team_conference_pre2002']
del file_df2['team_division_pre2002']



In [107]:
file_df2.head(5)

,team_name,team_id,team_conference,team_division
0,Arizona Cardinals,ARI,NFC,NFC West
1,Phoenix Cardinals,ARI,NFC,NaN
2,St. Louis Cardinals,ARI,NFC,NaN
3,Atlanta Falcons,ATL,NFC,NFC South
4,Baltimore Ravens,BAL,AFC,AFC North


In [109]:
df2= pd.DataFrame(file_df2)

df2.to_csv(r'C:\Users\Stron\csv_files\df2.csv', index = False, header=True)

In [72]:
# http://localhost:8889/edit/Resources/spreadspoke_scores.csv/spreadspoke_scores.csv

file3 = "Resources/spreadspoke_scores.csv/spreadspoke_scores.csv"

file_df3 = pd.read_csv(file3, encoding="ISO-8859-1")

file_df3.head(10)

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail,Unnamed: 17
0,9/2/1966,1966,1,False,Miami Dolphins,14.0,23.0,Oakland Raiders,NaN,NaN,NaN,Orange Bowl,False,83.0,6.0,71,NaN,NaN
1,9/3/1966,1966,1,False,Houston Oilers,45.0,7.0,Denver Broncos,NaN,NaN,NaN,Rice Stadium,False,81.0,7.0,70,NaN,NaN
2,9/4/1966,1966,1,False,San Diego Chargers,27.0,7.0,Buffalo Bills,NaN,NaN,NaN,Balboa Stadium,False,70.0,7.0,82,NaN,NaN
3,9/9/1966,1966,2,False,Miami Dolphins,14.0,19.0,New York Jets,NaN,NaN,NaN,Orange Bowl,False,82.0,11.0,78,NaN,NaN
4,9/10/1966,1966,1,False,Green Bay Packers,24.0,3.0,Baltimore Colts,NaN,NaN,NaN,Lambeau Field,False,64.0,8.0,62,NaN,NaN
5,9/10/1966,1966,2,False,Houston Oilers,31.0,0.0,Oakland Raiders,NaN,NaN,NaN,Rice Stadium,False,77.0,6.0,82,NaN,NaN
6,9/10/1966,1966,2,False,San Diego Chargers,24.0,0.0,New England Patriots,NaN,NaN,NaN,Balboa Stadium,False,69.0,9.0,81,NaN,NaN
7,9/11/1966,1966,1,False,Atlanta Falcons,14.0,19.0,Los Angeles Rams,NaN,NaN,NaN,Atlanta-Fulton County Stadium,False,71.0,7.0,57,NaN,NaN
8,9/11/1966,1966,2,False,Buffalo Bills,20.0,42.0,Kansas City Chiefs,NaN,NaN,NaN,War Memorial Stadium,False,63.0,11.0,73,NaN,NaN
9,9/11/1966,1966,1,False,Detroit Lions,14.0,3.0,Chicago Bears,NaN,NaN,NaN,Tiger Stadium,False,67.0,7.0,73,NaN,NaN


In [73]:
yearcount = len(file_df3["schedule_date"].unique())
yearcount

2462

In [74]:
earliest_year = file_df3["schedule_date"].min()
latest_year = file_df3["schedule_date"].max()
earliest_year

'1/1/1967'

In [75]:
latest_year

'9/9/2021'

In [76]:
file_df3.dtypes

schedule_date           object
schedule_season          int64
schedule_week           object
schedule_playoff          bool
team_home               object
score_home             float64
score_away             float64
team_away               object
team_favorite_id        object
spread_favorite        float64
over_under_line         object
stadium                 object
stadium_neutral         object
weather_temperature    float64
weather_wind_mph       float64
weather_humidity        object
weather_detail          object
Unnamed: 17            float64
dtype: object

In [77]:
file_df3['schedule_date'] = pd.to_datetime(file_df3.schedule_date)

In [78]:
file_df3.dtypes

schedule_date          datetime64[ns]
schedule_season                 int64
schedule_week                  object
schedule_playoff                 bool
team_home                      object
score_home                    float64
score_away                    float64
team_away                      object
team_favorite_id               object
spread_favorite               float64
over_under_line                object
stadium                        object
stadium_neutral                object
weather_temperature           float64
weather_wind_mph              float64
weather_humidity               object
weather_detail                 object
Unnamed: 17                   float64
dtype: object

In [87]:
pd.to_datetime("1/1/2021")

Timestamp('2021-01-01 00:00:00')

In [90]:
ts = pd.to_datetime("1/1/2021")

In [91]:
file_df3.loc[file_df3.schedule_date >= ts, :]

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail,Unnamed: 17
12918,2021-01-03,2020,17,False,Buffalo Bills,56.0,26.0,Miami Dolphins,MIA,-3.0,42.5,New Era Field,False,NaN,NaN,NaN,NaN,NaN
12919,2021-01-03,2020,17,False,Carolina Panthers,7.0,33.0,New Orleans Saints,NO,-6.0,46.5,Bank of America Stadium,False,NaN,NaN,NaN,NaN,NaN
12920,2021-01-03,2020,17,False,Chicago Bears,16.0,35.0,Green Bay Packers,GB,-4.5,48.5,Soldier Field,False,NaN,NaN,NaN,NaN,NaN
12921,2021-01-03,2020,17,False,Cincinnati Bengals,3.0,38.0,Baltimore Ravens,BAL,-13.5,45,Paul Brown Stadium,False,NaN,NaN,NaN,NaN,NaN
12922,2021-01-03,2020,17,False,Cleveland Browns,24.0,22.0,Pittsburgh Steelers,CLE,-10.5,44.5,FirstEnergy Stadium,False,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13182,2021-12-26,2021,16,False,New England Patriots,NaN,NaN,Buffalo Bills,NaN,NaN,NaN,Gillette Stadium,False,NaN,NaN,NaN,NaN,NaN
13183,2021-12-26,2021,16,False,New York Jets,NaN,NaN,Jacksonville Jaguars,NaN,NaN,NaN,MetLife Stadium,False,NaN,NaN,NaN,NaN,NaN
13184,2021-12-26,2021,16,False,Philadelphia Eagles,NaN,NaN,New York Giants,NaN,NaN,NaN,Lincoln Financial Field,False,NaN,NaN,NaN,NaN,NaN
13185,2021-12-26,2021,16,False,Seattle Seahawks,NaN,NaN,Chicago Bears,NaN,NaN,NaN,Lumen Field,False,NaN,NaN,NaN,NaN,NaN


In [92]:
new_df3 = file_df3.loc[file_df3.schedule_date >= ts, :]

In [95]:
new_df3.head()

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail,Unnamed: 17
12918,2021-01-03,2020,17,False,Buffalo Bills,56.0,26.0,Miami Dolphins,MIA,-3.0,42.5,New Era Field,False,NaN,NaN,NaN,NaN,NaN
12919,2021-01-03,2020,17,False,Carolina Panthers,7.0,33.0,New Orleans Saints,NO,-6.0,46.5,Bank of America Stadium,False,NaN,NaN,NaN,NaN,NaN
12920,2021-01-03,2020,17,False,Chicago Bears,16.0,35.0,Green Bay Packers,GB,-4.5,48.5,Soldier Field,False,NaN,NaN,NaN,NaN,NaN
12921,2021-01-03,2020,17,False,Cincinnati Bengals,3.0,38.0,Baltimore Ravens,BAL,-13.5,45,Paul Brown Stadium,False,NaN,NaN,NaN,NaN,NaN
12922,2021-01-03,2020,17,False,Cleveland Browns,24.0,22.0,Pittsburgh Steelers,CLE,-10.5,44.5,FirstEnergy Stadium,False,NaN,NaN,NaN,NaN,NaN


In [97]:
del new_df3['weather_temperature']
del new_df3['weather_wind_mph']
del new_df3['weather_humidity']
del new_df3['weather_detail']
del new_df3['Unnamed: 17']

In [101]:
new_df3.head(10)

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral
12918,2021-01-03,2020,17,False,Buffalo Bills,56.0,26.0,Miami Dolphins,MIA,-3.0,42.5,New Era Field,False
12919,2021-01-03,2020,17,False,Carolina Panthers,7.0,33.0,New Orleans Saints,NO,-6.0,46.5,Bank of America Stadium,False
12920,2021-01-03,2020,17,False,Chicago Bears,16.0,35.0,Green Bay Packers,GB,-4.5,48.5,Soldier Field,False
12921,2021-01-03,2020,17,False,Cincinnati Bengals,3.0,38.0,Baltimore Ravens,BAL,-13.5,45,Paul Brown Stadium,False
12922,2021-01-03,2020,17,False,Cleveland Browns,24.0,22.0,Pittsburgh Steelers,CLE,-10.5,44.5,FirstEnergy Stadium,False
12923,2021-01-03,2020,17,False,Denver Broncos,31.0,32.0,Las Vegas Raiders,LVR,-2.5,50.5,Sports Authority Field at Mile High,False
12924,2021-01-03,2020,17,False,Detroit Lions,35.0,37.0,Minnesota Vikings,MIN,-3.0,53.5,Ford Field,False
12925,2021-01-03,2020,17,False,Houston Texans,38.0,41.0,Tennessee Titans,TEN,-7.0,55.5,NRG Stadium,False
12926,2021-01-03,2020,17,False,Indianapolis Colts,28.0,14.0,Jacksonville Jaguars,IND,-15.5,48,Lucas Oil Stadium,False
12927,2021-01-03,2020,17,False,Kansas City Chiefs,21.0,38.0,Los Angeles Chargers,LAC,-6.5,43,Arrowhead Stadium,False


In [106]:
df3= pd.DataFrame(new_df3)

df3.to_csv(r'C:\Users\Stron\csv_files.csv', index = False, header=True)
